In [1]:
import sys

sys.path.append("..")

from core.DataLoader import DataPreprocessor, get_load_config_from_yaml
from importlib import reload
import matplotlib.pyplot as plt
import os


if False:
    PLOTS_DIR = "plots/ToponiumInfusedTransformer/"
    MODEL_DIR = "models/mixed_model"
elif True:
    PLOTS_DIR = "plots/RegressionTransformer_nu_flows_neutrino_only/"
    MODEL_DIR = "../models/regression_transformer_pil/"
else:
    PLOTS_DIR = "plots/RegressionTransformer_/"
    MODEL_DIR = "../models/regression_transformer_old/"


CONFIG_PATH = "../config/workspace_config.yaml"

# Create directories
os.makedirs(PLOTS_DIR, exist_ok=True)

# Set plotting style
plt.rcParams.update({"font.size": 18})

# Load data
load_config = get_load_config_from_yaml(CONFIG_PATH)
DataProcessor = DataPreprocessor(load_config)
data_config = DataProcessor.load_from_npz(
    load_config.data_path["nominal"], event_numbers="odd"
)
PREDICTIONS_DIR = os.path.join(load_config.data_dir, "RegressionTransformer_nu_flows_neutrino_only", "odd_predictions/")
os.makedirs(PREDICTIONS_DIR, exist_ok=True)

X, y = DataProcessor.get_data()
del DataProcessor

2026-01-28 14:03:44.049512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769605424.070902  335265 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769605424.077866  335265 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769605424.095293  335265 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769605424.095310  335265 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769605424.095313  335265 computation_placer.cc:177] computation placer alr

In [2]:
import core.reconstruction.BaselineMethods as BaselineMethods
import core.evaluation.reconstruction_evaluator as Evaluation
from core.reconstruction import (
    GroundTruthReconstructor,
    PerfectAssignmentReconstructor,
    CompositeNeutrinoComponentReconstructor,
)

reload(Evaluation)
reload(BaselineMethods)
ground_truth_assigner = GroundTruthReconstructor(data_config, use_nu_flows=True)
perfect_assigner = PerfectAssignmentReconstructor(
    data_config, neutrino_reco_name=r"PIL Transformer $\nu$"
)
perfect_assigner.load_model(f"{MODEL_DIR}/model.keras")
composite_neutrino_predictor = CompositeNeutrinoComponentReconstructor(
    data_config, use_nu_flows=True, axis=(-1), name=r"PIL Transformer $\nu$"
)
composite_neutrino_predictor.load_model(f"{MODEL_DIR}/model.keras")

prediction_manager = Evaluation.PredictionManager(
    [ground_truth_assigner, perfect_assigner], X, y
)


prediction_manager.save_predictions(PREDICTIONS_DIR)

I0000 00:00:1769605607.892056  335265 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12977 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


TypeError: <class 'core.base_classes.machine_learning_base.KerasModelWrapper'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'core.base_classes.machine_learning_base', 'class_name': 'KerasModelWrapper', 'config': {}, 'registered_name': 'Custom>KerasModelWrapper', 'build_config': {'input_shape': None}, 'compile_config': {}}.

Exception encountered: Could not locate class 'OutputUpScaleLayer'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'core.components.output_layers', 'class_name': 'OutputUpScaleLayer', 'config': {'name': 'regression', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None, 'shared_object_id': 140268667830576}}, 'registered_name': '